In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

In [49]:
from fosforio import snowflake
from fosforio import get_dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#import seaborn as sns
#from imblearn.over_sampling import SMOTE
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib

In [5]:
snowflake.get_connection(connection_name="HR Attrition conn")
df = get_dataframe("Final_HR_Attrition_Data")
df

Existing connection object fetched: <snowflake.connector.connection.SnowflakeConnection object at 0x7f84183d8040>
Please close the connection after use!
Reading dataframe from snowflake native connector


,SALARY,SENIORITY,TENURE_MONTHS,MONTHS_AFTER_COLLEGE,BIRTH_YEAR,AGE,MAPPED_ROLE_CLEAN,SEX,ETHNICITY,HOSPITAL_TYPE,...,OVER_TIME_HOURS,CHURN,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION,JOB_SATISFACTION,MARTIAL_STATUS,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,RELATIONSHIP_SATISFACTION,WORK_LIFE_BALANCE
0,68271.546,3,8,116,1984,40,occupational,F,Black,Childrens,...,15,False,Travel_Rarely,High,Low,Married,11,Excellent,Medium,Best
1,70471.235,3,37,18,1992,32,nurse,F,Hispanic,Psychiatric,...,10,True,Travel_Rarely,High,Medium,Married,11,Outstanding,Low,Good
2,53490.238,2,47,16,1988,36,emt,F,White,Psychiatric,...,14,False,Travel_Rarely,Very High,Low,Single,19,Good,Low,Bad
3,47749.212,1,40,32,1983,41,nurse,M,Hispanic,Acute Care Hospitals,...,3,False,Travel_Frequently,High,High,Single,13,Low,Medium,Bad
4,61379.954,1,29,11,1992,32,occupational,F,Hispanic,Psychiatric,...,1,True,Travel_Frequently,High,Medium,Married,19,Low,Medium,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,69870.670,2,42,14,1995,29,nurse,M,Black,Acute Care - Department of Defense,...,6,False,Travel_Rarely,Medium,Very High,Divorced,13,Excellent,Low,Good
299996,79047.028,5,106,164,1975,49,technologist,M,Hispanic,Acute Care - Department of Defense,...,6,True,Travel_Rarely,Low,Medium,Divorced,14,Excellent,Medium,Best
299997,51505.375,1,12,69,1985,39,social,F,Multiple,Acute Care - Department of Defense,...,9,False,Travel_Rarely,Low,Medium,Married,18,Good,Very High,Best
299998,64988.218,2,9,76,1985,39,occupational,F,API,Childrens,...,14,False,Non-Travel,Very High,Very High,Single,15,Outstanding,Low,Better


In [6]:
print(df.isnull().sum())

SALARY                           0
SENIORITY                        0
TENURE_MONTHS                    0
MONTHS_AFTER_COLLEGE             0
BIRTH_YEAR                       0
AGE                              0
MAPPED_ROLE_CLEAN                0
SEX                              0
ETHNICITY                        0
HOSPITAL_TYPE                    0
HOSPITAL_OWNERSHIP               0
COMPANY_NAME                     0
CITY                             0
STATE                            0
DISTANCE                         0
DEGREE_CLEAN                     0
SCHOOL_END_DATE                  0
JOB_START_DATE                   0
JOB_END_DATE                 90102
USER_ID                          0
OVER_TIME_HOURS                  0
CHURN                            0
BUSINESS_TRAVEL                  0
ENVIRONMENT_SATISFACTION         0
JOB_SATISFACTION                 0
MARTIAL_STATUS                   0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
RELATIONSHIP_SATISFA

In [7]:
df = df.dropna()

In [8]:
print(df.isnull().sum())

SALARY                       0
SENIORITY                    0
TENURE_MONTHS                0
MONTHS_AFTER_COLLEGE         0
BIRTH_YEAR                   0
AGE                          0
MAPPED_ROLE_CLEAN            0
SEX                          0
ETHNICITY                    0
HOSPITAL_TYPE                0
HOSPITAL_OWNERSHIP           0
COMPANY_NAME                 0
CITY                         0
STATE                        0
DISTANCE                     0
DEGREE_CLEAN                 0
SCHOOL_END_DATE              0
JOB_START_DATE               0
JOB_END_DATE                 0
USER_ID                      0
OVER_TIME_HOURS              0
CHURN                        0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
JOB_SATISFACTION             0
MARTIAL_STATUS               0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
RELATIONSHIP_SATISFACTION    0
WORK_LIFE_BALANCE            0
dtype: int64


In [11]:
df = df.drop(["USER_ID", "JOB_START_DATE", "JOB_END_DATE", "SCHOOL_END_DATE"], axis = 1)

In [67]:
CATEGORICAL_COLUMNS = ["MAPPED_ROLE_CLEAN","SEX", "ETHNICITY","HOSPITAL_TYPE", "HOSPITAL_OWNERSHIP","COMPANY_NAME","CITY","STATE","DISTANCE", 
                       "DEGREE_CLEAN","BUSINESS_TRAVEL","ENVIRONMENT_SATISFACTION","JOB_SATISFACTION","MARTIAL_STATUS","PERFORMANCE_RATING","RELATIONSHIP_SATISFACTION","WORK_LIFE_BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR","AGE", "OVER_TIME_HOURS", "PERCENTAGE_SALARY_HIKE",]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [68]:
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS 

In [69]:
feature_columns = [col for col in feature_columns if col in df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in df.columns]

In [70]:
X = df[feature_columns]
Y = df[LABEL_COLUMNS]

In [71]:
X.head()

,MAPPED_ROLE_CLEAN,SEX,ETHNICITY,HOSPITAL_TYPE,HOSPITAL_OWNERSHIP,COMPANY_NAME,CITY,STATE,DISTANCE,DEGREE_CLEAN,...,RELATIONSHIP_SATISFACTION,WORK_LIFE_BALANCE,SALARY,SENIORITY,TENURE_MONTHS,MONTHS_AFTER_COLLEGE,BIRTH_YEAR,AGE,OVER_TIME_HOURS,PERCENTAGE_SALARY_HIKE
0,occupational,F,Black,Childrens,Department of Defense,Village Practice Management Co. LLC,Lewiston,ME,>10 Miles,Professional Certificate,...,Medium,Best,68271.546,3,8,116,1984,40,15,11
1,nurse,F,Hispanic,Psychiatric,Proprietary,McKesson Corp.,Charlottesville,VA,>10 Miles,Undergraduate Diploma,...,Low,Good,70471.235,3,37,18,1992,32,10,11
2,emt,F,White,Psychiatric,Government - Hospital District or Authority,24-7 Bright Star Healthcare LLC,Evansville,IN,2-5 Miles,Masters Degree,...,Low,Bad,53490.238,2,47,16,1988,36,14,19
3,nurse,M,Hispanic,Acute Care Hospitals,Government - Hospital District or Authority,AmerisourceBergen Corp.,Lakeland,FL,<2 Miles,Undergraduate Diploma,...,Medium,Bad,47749.212,1,40,32,1983,41,3,13
4,occupational,F,Hispanic,Psychiatric,Government - Federal,Cleveland Clinic,Buffalo,NY,5-10 Miles,Doctoral Degree,...,Medium,Bad,61379.954,1,29,11,1992,32,1,19


In [72]:
Y.head()

,CHURN
0,False
1,True
2,False
3,False
4,True


In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.20, random_state = 42)

In [74]:
## train_df and test_df are both snowpark dataframes
pipeline = Pipeline(
    steps=[(
                "OE",
                OrdinalEncoder(
                input_cols=CATEGORICAL_COLUMNS,
                output_cols=CATEGORICAL_COLUMNS,
                handle_unknown='use_encoded_value',
                unknown_value=-1
                )),
               ("MMS",
                MinMaxScaler(
                clip=True,
                input_cols=NUMERICAL_COLUMNS,
                output_cols=NUMERICAL_COLUMNS,
                )),
               ("classification",
                XGBClassifier(
                input_cols=CATEGORICAL_COLUMNS+NUMERICAL_COLUMNS,
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                ))])

pipeline.fit(X_train) ## fiting the dataset
result = pipeline.predict(X_test) 

TypeError: __init__() got an unexpected keyword argument 'input_cols'

In [51]:
# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), CATEGORICAL_COLUMNS),
        ('num', MinMaxScaler(), NUMERICAL_COLUMNS)
    ]
)

In [63]:
# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])
 

In [66]:
# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

ValueError: A given column is not a column of the dataframe